# Imports

In [7]:
# imports
import glob
import re
from pathlib import Path
import pandas as pd
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt
import torch
# import numpy as np
# import matplotlib.pyplot as plt
# from scipy.spatial import distance
# from sklearn.model_selection import train_test_split
from skimage import measure, filters
from skimage.filters import gaussian
import tensorflow as tf
import random
# import os
import torchvision.transforms as transforms
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Mount drive

In [8]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Unzip data from Google Drive

In [11]:
# from google.colab import files
# uploaded = files.upload()
import zipfile

!unzip /content/drive/MyDrive/CS682/Project/archive.zip

Streaming output truncated to the last 5000 lines.
  inflating: images_gz2/images/95074.jpg  
  inflating: images_gz2/images/95075.jpg  
  inflating: images_gz2/images/95076.jpg  
  inflating: images_gz2/images/95077.jpg  
  inflating: images_gz2/images/95078.jpg  
  inflating: images_gz2/images/95079.jpg  
  inflating: images_gz2/images/9508.jpg  
  inflating: images_gz2/images/95080.jpg  
  inflating: images_gz2/images/95081.jpg  
  inflating: images_gz2/images/95082.jpg  
  inflating: images_gz2/images/95083.jpg  
  inflating: images_gz2/images/95084.jpg  
  inflating: images_gz2/images/95085.jpg  
  inflating: images_gz2/images/95086.jpg  
  inflating: images_gz2/images/95087.jpg  
  inflating: images_gz2/images/95088.jpg  
  inflating: images_gz2/images/95089.jpg  
  inflating: images_gz2/images/9509.jpg  
  inflating: images_gz2/images/95090.jpg  
  inflating: images_gz2/images/95091.jpg  
  inflating: images_gz2/images/95092.jpg  
  inflating: images_gz2/images/95093.jpg  
  inf

# Preprocess data

In [16]:
# follow the directory structure specified in the repository
# downloaded heart16 file from https://data.galaxyzoo.org/#section-7
df_hart16 = pd.read_csv('/content/gz2_hart16.csv')
# print("HART16 Dataset Info:")
# df_hart16.info()

# downloaded images and mapping from https://www.kaggle.com/datasets/jaimetrickz/galaxy-zoo-2-images/data
df_mappings = pd.read_csv('/content/gz2_filename_mapping.csv')
# print("\nMappings Dataset Info:")
# df_mappings.info()

# only keep first 4 columns and debiased data
columns_to_keep = ["dr7objid", "ra", "dec", "gz2_class"] + [col for col in df_hart16.columns if col.endswith('_debiased')]
filtered_hart16 = df_hart16[columns_to_keep]

# merge dataframes on object id
df_merged = pd.merge(df_mappings, filtered_hart16,
                     left_on='objid',
                     right_on='dr7objid')

# check for null rows
null_rows = df_merged.isnull().any(axis=1)
# get total number of null rows
num_null_rows = null_rows.sum()
# drop null rows
df_merged.dropna(inplace=True)

print(df_merged.shape)

# get list of all images in the image folder
image_files = glob.glob('/content/images_gz2/images*.jpg')
print("Image count:", len(image_files))

# regex to match numeric image file names
regex = re.compile(r".*[/\\](\d+)\.jpg")

# extract image names
# image name corresponds to assed_id in data table
image_names = []
for img in image_files:
    match = re.search(regex, img)
    if match:
        image_names.append(int(match.group(1)))

# sort and display
image_names.sort()
# print("First 10 image names:", image_names[:10])

print(df_merged.head())

class_counts = df_merged['gz2_class'].value_counts()
print("Number of classes:", class_counts.shape[0])

(239389, 44)
Image count: 0
                objid    sample  asset_id            dr7objid          ra  \
0  587722981741363294  original         3  587722981741363294  182.925262   
1  587722981741363323  original         4  587722981741363323  182.970108   
2  587722981741559888  original         5  587722981741559888  183.438095   
3  587722981741625481  original         6  587722981741625481  183.473999   
4  587722981741625484  original         7  587722981741625484  183.477783   

        dec gz2_class  t01_smooth_or_features_a01_smooth_debiased  \
0 -1.092357        Sb                                       0.459   
1 -1.219537      Sc?l                                       0.216   
2 -1.238414        Er                                       0.773   
3 -1.231429      Sc1t                                       0.152   
4 -1.084604        Sb                                       0.182   

   t01_smooth_or_features_a02_features_or_disk_debiased  \
0                                  

Augment images

In [26]:
from torchvision.utils import save_image
import os

# define paths to input and output folders
input_folder = '/content/images_gz2/images'
output_folder = '/content/images_gz2/processed_images'

# define transformation function
def compute_train_transform(seed=123456):
    random.seed(seed)
    torch.random.manual_seed(seed)

    colour_jitter = transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)
    train_transform = transforms.Compose([
        # randomly resize and crop to 32 x 32
        transforms.RandomResizedCrop(32),
        # horizontal flip with 50% probability
        transforms.RandomHorizontalFlip(p=0.5),
        # colour jitter with 80% probability
        transforms.RandomApply([colour_jitter], p=0.8),
        # convert to greyscale with 20% probability
        transforms.RandomGrayscale(p=0.2),
        # convert image to tensor
        transforms.ToTensor(),
    ])
    return train_transform

# # load image for testing
# image_path = "/content/images_gz2/images/223272.jpg"  # Update with your image path
# img = Image.open(image_path)

# train_transform = compute_train_transform()
# transformed_img_tensor = train_transform(img)

# transformed_img = transforms.ToPILImage()(transformed_img_tensor)

# # display the processed image
# plt.imshow(transformed_img)
# plt.axis('off')
# plt.show()


# ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# loop over all images in the input folder
for filename in os.listdir(input_folder):
    img_path = os.path.join(input_folder, filename)

    # skip non-image files
    if not filename.endswith(('.jpg', '.jpeg', '.png')):
        continue

    # load image
    img = Image.open(img_path)

    # apply transformation
    train_transform = compute_train_transform()
    img_transformed = train_transform(img)

    # save transformed image to output folder
    output_path = os.path.join(output_folder, f'processed_{filename}')
    save_image(img_transformed, output_path)

    print(f"Saved processed image: {output_path}")

Streaming output truncated to the last 5000 lines.
Saved processed image: /content/images_gz2/processed_images/processed_273021.jpg
Saved processed image: /content/images_gz2/processed_images/processed_54769.jpg
Saved processed image: /content/images_gz2/processed_images/processed_137406.jpg
Saved processed image: /content/images_gz2/processed_images/processed_246024.jpg
Saved processed image: /content/images_gz2/processed_images/processed_246046.jpg
Saved processed image: /content/images_gz2/processed_images/processed_226290.jpg
Saved processed image: /content/images_gz2/processed_images/processed_174111.jpg
Saved processed image: /content/images_gz2/processed_images/processed_1089.jpg
Saved processed image: /content/images_gz2/processed_images/processed_69605.jpg
Saved processed image: /content/images_gz2/processed_images/processed_180046.jpg
Saved processed image: /content/images_gz2/processed_images/processed_20506.jpg
Saved processed image: /content/images_gz2/processed_images/pro

Split data into train and test sets

In [28]:
from sklearn.model_selection import train_test_split
import shutil
import os

# define paths to images
input_folder = '/content/images_gz2/processed_images'
train_folder = '/content/images_gz2/train_images'
test_folder = '/content/images_gz2/test_images'

# create train and test directories if they don't exist
os.makedirs(train_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

# get all image filenames
all_images = [f for f in os.listdir(input_folder) if f.endswith(('.jpg', '.jpeg', '.png'))]

# split the images into train and test sets
train_images, test_images = train_test_split(all_images, test_size=0.2, random_state=42)

# move images to train directory
for image in train_images:
    shutil.move(os.path.join(input_folder, image), os.path.join(train_folder, image))

# move images to test directory
for image in test_images:
    shutil.move(os.path.join(input_folder, image), os.path.join(test_folder, image))

print(f"Training images: {len(train_images)}, Test images: {len(test_images)}")


Training images: 182024, Test images: 45506


# Define CNN

In [29]:
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            # adjust based on image size after convolutions
            nn.Linear(64 * 32 * 32, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

Train CNN on images

In [33]:
import torch.optim as optim
from torchvision import datasets
from torch.utils.data import DataLoader

# find number of classes
num_classes = df_merged['gz2_class'].value_counts()
print("Number of classes:", num_classes.shape[0])

# train model
model = CNN(num_classes.shape[0])
criterion = nn.CrossEntropyLoss()
optim = optim.Adam(model.parameters(), lr=0.001)

# load datasets
train_dataset = datasets.ImageFolder(root='/content/images_gz2/train_images', transform=transform)
val_dataset = datasets.ImageFolder(root='/content/images_gz2/test_images', transform=transform)

# create DataLoader for batching data
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for i, (inputs, labels) in enumerate(train_loader, 0):
        # move to GPU if available
        inputs, labels = inputs.cuda(), labels.cuda() if torch.cuda.is_available() else (inputs, labels)

        # zero parameter gradients
        optim.zero_grad()

        # forward pass
        outputs = model(inputs)

        # calculate loss
        loss = criterion(outputs, labels)

        # backpropagate and optimise
        loss.backward()
        optim.step()

        # print statistics
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # print loss and accuracy for the epoch
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")


Number of classes: 818


FileNotFoundError: Couldn't find any class folder in /content/images_gz2/train_images.